In [1]:
import cv2
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import math
import numpy as np
import os

In [2]:
AUDIO_RATE = 16000
y1, y2, y3, y4, x1, x2, x3, x4 = (128, 352, 128, 352, 68, 292, 428, 652)

In [3]:
import pims

In [4]:
def gen_video_array(vid):
    sess_id = vid[4]
    video_path = "E:/datasets/IEMOCAP_full_release.tar/IEMOCAP_full_release/IEMOCAP_full_release/Session{}/dialog/avi/DivX/{}.avi".format(sess_id,vid)
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    success=True
    frame_list = []
    while success:
        success, frame=cap.read()
        frame_list.append(frame)

    video_array = np.array(frame_list[:-1])
    del frame_list
    return video_array, fps

In [5]:
def save_arr2vid(arr, output_path, fps):
    out = cv2.VideoWriter(output_path, 
                          cv2.VideoWriter_fourcc(*'mp4v'), 
                          fps, (arr.shape[2], arr.shape[1]))
    for i in range(arr.shape[0]):
        data = arr[i, :, :, :]
        out.write(data)
    out.release()

In [6]:
def select_face(faces, cached_center, arr_shape):
    width = height = 88
    if len(faces) == 0:
        ret_face = [cached_center[0]-width//2, cached_center[1]-height//2, width, height]
    else:
        min_dist = np.inf
        for face in faces:
            x,y,w,h = face
            xc = int(round((2*x+w)/2))
            yc = int(round((2*y+h)/2))
            
            xc = max(min(xc, arr_shape[0]-width//2), width//2)
            yc = max(min(yc, arr_shape[1]-height//2), height//2)
            
            dist = abs(xc-cached_center[0])+abs(yc-cached_center[1])
            if dist < min_dist:
                min_dist = dist
                ret_face = xc-w//2, yc-h//2, w, h
            ### Stablize
            if min_dist < 20:
                ret_face = [cached_center[0]-width//2, cached_center[1]-height//2, width, height]
    return ret_face

def save_video_array(vid, output_dir, fps):
    sess_id = vid[4]
    video_path = "E:/datasets/IEMOCAP_full_release.tar/IEMOCAP_full_release/IEMOCAP_full_release/Session{}/dialog/avi/DivX/{}.avi".format(sess_id,vid)
    video = pims.Video(video_path)
    df = pd.read_excel('E:/datasets/preprocessed/extractionmap/cut_extractionmap{}.xlsx'.format(sess_id), sheet_name=vid)
    df["viframe"] = np.floor(df["iframe"]*fps/AUDIO_RATE).astype(int)
    df["vfframe"] = np.ceil(df["fframe"]*fps/AUDIO_RATE).astype(int)
    
    face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    width = height = 88
    
    cached_center_l = None
    cached_center_r = None
    
    for index, row in df.iterrows():
        varr = np.asarray(video[row["viframe"]:row["vfframe"]])
        if row["speaker"] == "L":
            arr = varr[:, 
                       y1:y2, 
                       x1:x2, :]
            if cached_center_l == None:
                cached_center_l = [arr.shape[2]//2, arr.shape[1]//2]
                
        elif row["speaker"] == "R":
            arr = varr[:, 
                       y3:y4, 
                       x3:x4, :]
            if cached_center_r == None:
                cached_center_r = [arr.shape[2]//2, arr.shape[1]//2]
            
        img_stack_sm = []
        

        for idx in range(arr.shape[0]):
            img = arr[idx, :, :, :]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray)
            if row["speaker"] == "L":
                face = select_face(faces, cached_center_l, (arr.shape[1], arr.shape[2]))
                x,y,w,h = face
                xc = int(round((2*x+w)/2))
                yc = int(round((2*y+h)/2))
                cached_center_l = [xc, yc]
                
            elif row["speaker"] == "R":
                face = select_face(faces, cached_center_r, (arr.shape[1], arr.shape[2]))
                x,y,w,h = face
                xc = int(round((2*x+w)/2))
                yc = int(round((2*y+h)/2))
                cached_center_r = [xc, yc]
            
#             print(gray.shape)
#             print(xc, yc)
#             print(yc-int(height/2), yc+int(height/2), 
#                             xc-int(width/2), xc+int(width/2))
            face_img = gray[yc-int(height/2):yc+int(height/2), 
                            xc-int(width/2):xc+int(width/2)]
            face_img = cv2.cvtColor(face_img, cv2.COLOR_GRAY2BGR)
            face_img = cv2.resize(face_img, (88, 88), interpolation=cv2.INTER_AREA)
            img_stack_sm.append(face_img)
            
        img_stack_sm = np.array(img_stack_sm)
        save_path = os.path.join(output_dir, "{}".format(row["smp_id"]))
        print(save_path)
        save_arr2vid(img_stack_sm, save_path+".mp4", fps)

In [8]:
# arr = save_video_array(smp_id_list[1], "test", 30)

### Pipeline

In [9]:
import torch
import sys
sys.path.append("E:/university/FYT/repos/multi_modal_ser")
from utils.dataset import MMSERDataset
mmser_ds = torch.load("E:/datasets/preprocessed/dataset/mmser_ds.pt")
df_ = mmser_ds.df_

In [10]:
df_["fn"] = df_["smp_id"].apply(lambda x: x[:-5])
smp_id_list = df_["fn"].unique().tolist()

In [11]:
smp_id_list = [i for i in smp_id_list if not i.startswith("Ses01") ]

In [12]:
vid = smp_id_list[0]
save_video_array(vid,
                 "E:/datasets/preprocessed/roi/{}".format(vid.split("_")[0][:-1]),
                 30)

E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F000
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_M000
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_M001
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F001
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_M002
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F002
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_M003
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F003
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F004
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F005
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_M004
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_M005
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F006
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_M006
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F007
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_M007
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F008
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F009
E:/dataset

In [13]:
from tqdm import tqdm 

fps=30
for vid in tqdm(smp_id_list):
    print(vid)
    save_video_array(vid,
                 "E:/datasets/preprocessed/roi/{}".format(vid.split("_")[0][:-1]),
                 fps)

  0%|                                                                                                                                    | 0/123 [00:00<?, ?it/s]

Ses02F_impro01
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F000
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_M000
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_M001
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F001
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_M002
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F002
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_M003
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F003
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F004
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F005
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_M004
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_M005
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F006
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_M006
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F007
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_M007
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F008
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_

  1%|▉                                                                                                                         | 1/123 [01:17<2:37:09, 77.29s/it]

E:/datasets/preprocessed/roi/Ses02\Ses02F_impro01_F023
Ses02F_impro02
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro02_F000
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro02_M000
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro02_F001
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro02_M001
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro02_F002
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro02_M002
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro02_F003
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro02_M003
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro02_F004
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro02_F005
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro02_M004
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro02_M005
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro02_F006
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro02_F007
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro02_M006
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro02_F008
E:/datasets/preprocessed/roi/Ses02\Ses02F_impro02_

  2%|█▉                                                                                                                        | 2/123 [02:12<2:10:02, 64.48s/it]

...

E:/datasets/preprocessed/roi/Ses05\Ses05M_script02_2_M040
Ses05M_script03_1
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_1_M000
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_1_F000
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_1_M001
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_1_F001
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_1_M002
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_1_F002
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_1_M003
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_1_F003
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_1_M004
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_1_F004
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_1_M005
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_1_F005
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_1_M006
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_1_F006
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_1_M007
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_1_F

 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████ | 122/123 [35:32:29<2:43:16, 9796.16s/it]

E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_1_F031
Ses05M_script03_2
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_2_M000
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_2_F000
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_2_M001
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_2_F001
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_2_M002
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_2_F002
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_2_M003
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_2_F003
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_2_M004
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_2_F004
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_2_M005
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_2_F005
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_2_M006
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_2_F006
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_2_M007
E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_2_F

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [35:37:33<00:00, 1042.71s/it]

E:/datasets/preprocessed/roi/Ses05\Ses05M_script03_2_F044


In [ ]:
# save_video_array(smp_id_list[0],
#                  "E:/datasets/preprocessed/face_roi/{}".format(smp_id_list[0].split("_")[0][:-1]),
#                  30)